In [1]:
# Guide to Build Video Classification Model
# https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/

In [2]:
!pip3 install opencv-python

In [3]:
# Importing required libraries

import os
import cv2  # for caputring videos
import math # for mathematical operations
import matplotlib.pyplot as plt # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image # for preprocessing the images
import numpy as np # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [4]:
#Storing the name of videos in a dataframe

# open the .txt file which have names of training videos
f = open("trainlist_tennis.txt", "r")
temp = f.read()
videos = temp.split('\n')


# creating a dataframe having video names
train = pd.DataFrame()
train['video_name'] = videos
# train = train[:-1] #grabs the last one 


In [5]:
#Create a similar dataframe for test videos

# open the .txt file which have names of test videos
f = open("testlist_tennis.txt", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
# test = test[:-1]
test

,video_name
0,4_0.mp4
1,5_0.mp4
2,6_1.mp4


In [6]:
#Add the tag of each video (for both training and test sets)
#Entire part before the '/' in the video name represents the tag of the video
#Split the entire string on '/' and select the tag for all the videos

# creating tags for training videos
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('_')[1].split('.')[0])
    
train['tag'] = train_video_tag

# creating tags for test videos
test_video_tag = []
for i in range(test.shape[0]):
    test_video_tag.append(test['video_name'][i].split('_')[1].split('.')[0])
    
test['tag'] = test_video_tag

In [7]:
#This is what the test dataframe looks like 
test

,video_name,tag
0,4_0.mp4,0
1,5_0.mp4,0
2,6_1.mp4,1


In [8]:
# This is what the train dataframe looks like 
train

,video_name,tag
0,1_0.mp4,0
1,2_0.mp4,0
2,3_1.mp4,1


In [9]:
#Extract the frames from the training videos which will be used to train the model
#Store all the frames in a folder named train_1

# storing the frames from training videos
for i in tqdm(range(train.shape[0])):
    count = 0
    videoFile = train['video_name'][i]
    cap = cv2.VideoCapture('myVids/'+videoFile)   # capturing the video from the given path
    frameRate = cap.get(5) #frame rate
    x=1
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        ret, frame = cap.read()
        if (ret != True):
            break
        if (frameId % math.floor(frameRate) == 0):
            # storing the frames in a new folder named train_1
            print("this is the split")
            print(videoFile.split(" ")[0]) #since my videos aren't in separate folder, no need to split based on ("/")
            
            #need to make sure that the train_1 has been created
            filename ='train_1/' + videoFile.split(' ')[0] +"_frame%d.jpg" % count;count+=1
            cv2.imwrite(filename, frame)
    cap.release()

  0%|          | 0/3 [00:00<?, ?it/s]

this is the split
1_0.mp4
this is the split
1_0.mp4
this is the split
1_0.mp4
this is the split
1_0.mp4


 33%|███▎      | 1/3 [00:00<00:01,  1.48it/s]

this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4
this is the split
2_0.mp4


 67%|██████▋   | 2/3 [00:05<00:01,  1.85s/it]

this is the split
2_0.mp4
this is the split
3_1.mp4
this is the split
3_1.mp4
this is the split
3_1.mp4
this is the split
3_1.mp4
this is the split
3_1.mp4
this is the split
3_1.mp4


100%|██████████| 3/3 [00:06<00:00,  2.13s/it]

this is the split
3_1.mp4
this is the split
3_1.mp4


In [15]:
#Save the name of these frames with their corresponding tag in a .csv file

#getting the names of all the images
images = glob("train_1/*.jpg")
train_image = []
train_class = []


for i in tqdm(range(len(images))):
    # creating the image name
    train_image.append(images[i].split("/")[1])
    # creating the class of image
    train_class.append(images[i].split("/")[1].split(".")[0].split("_")[1])
    
# storing the images and their class in a dataframe
train_data = pd.DataFrame()
train_data['image'] = train_image
train_data['class'] = train_class

# converting the dataframe into csv file 
train_data.to_csv('myVids/train_new.csv',header=True, index=False)

100%|██████████| 37/37 [00:00<00:00, 86843.45it/s]
